# Uva API to CSV

UVa has an api end point at [Uhunt](http://uhunt.felix-halim.net/api). This python script will extract information from api end point and convert to CSV files for Data Analysis

In [1]:
%load_ext Cython
%load_ext line_profiler

import requests as rq
import pandas as pd
from IPython.display import HTML, display
import os.path as path
import time
import datetime

def header(s):
    display( HTML('<h1 align="center">{}</h1>'.format(s)) )
    
def subheader(s):
    display( HTML('<h3 align="center">{}</h3>'.format(s)) )


In [11]:
%%cython

cpdef str toUUID( int x):
    return "00000000-0000-0000-0000-{:012}".format(x)

# UVa Problem List

First, lets grab all the problems on UVa

In [27]:
if path.exists('problem.csv'):
    # Already stored in disk
    problemDF = pd.read_csv('problem.csv')
else: 
    # Grab it from API
    problemList = rq.get( 'http://uhunt.felix-halim.net/api/p' ).json()
    columns = [ 
        'id', 
        'number', 
        'title',
        'dacu',
        'best_runtime',
        'dummy',
        'best_memory',
        'no_verdict',
        'submission_error', 
        'cannot_judge',
        'in_queue',
        'CE', # Compilation Error
        'RF', # Restricted Function
        'RTE',
        'OLE',
        'TLE',
        'MLE',
        'WA',
        'PE',
        'AC',
        'runtime',
        'status'
    ]
    problemDF = pd.DataFrame.from_records(problemList, columns=columns)
    del problemDF['dummy']
    problemDF.to_csv('problem.csv', index=False)
    
display ( problemDF.head() )

totalProblem = len(problemDF)
print ( "Total number of problems = {}".format(totalProblem))

,id,number,title,dacu,best_runtime,best_memory,no_verdict,submission_error,cannot_judge,in_queue,...,RF,RTE,OLE,TLE,MLE,WA,PE,AC,runtime,status
0,36,100,The 3n + 1 problem,79081,0,0,6949,0,0,112475,...,67231,321,58214,5209,272129,4988,193455,3000,1,0
1,37,101,The Blocks Problem,13246,0,0,933,0,0,13179,...,20724,18,9218,200,22623,6102,19991,3000,1,0
2,38,102,Ecological Bin Packing,23568,0,0,1972,0,0,12532,...,4813,28,3089,72,34729,669,36303,3000,1,0
3,39,103,Stacking Boxes,6209,0,0,0,0,0,11331,...,6188,2,2351,0,11007,0,10133,3000,2,0
4,40,104,Arbitrage,4022,0,0,310,0,0,2374,...,2314,16,3712,427,12689,593,6904,3000,2,0


Total number of problems = 4819


# Grab All Submissions from 2016

In order to grab all submissions in 2016, we will be using the following api: `/api/p/subs/{pids-csv}/{start-sbt}/{end-sbt}.` For this, we will be requiring problem-ids as csv and start-end time as unix time stamp

In [4]:
# Grab start and end time in unix time stamp
# date -d "Jan 1 2016" +%s 
start2016 = 1451584800
end2016 = 1483120800

problemIdList = problemDF['id'].values

subProblemIdList = problemIdList
totalSubProblem = len(subProblemIdList) - 1

problemSubFolder = 'problem_sub_2016'

for ind, problemId in enumerate ( subProblemIdList ) :
    fileName = path.join( problemSubFolder, "{}.csv".format(problemId) )
    if path.exists(fileName) :
        print ( "{:5}/{:<5} Done Processing Problem Id: {}. Already exists.".format(ind, totalSubProblem, problemId) )
    else :
        subAPI = "http://uhunt.felix-halim.net/api/p/subs/{}/{}/{}".format(problemId, start2016, end2016)
        sub = rq.get(subAPI).json()
        subDF = pd.DataFrame.from_records(sub)
        subDF.to_csv(fileName, index=False )
        print ( "{:5}/{:<5} Done Processing Problem Id: {}".format(ind, totalSubProblem, problemId) )

    0/4818  Done Processing Problem Id: 36. Already exists.
    1/4818  Done Processing Problem Id: 37. Already exists.
    2/4818  Done Processing Problem Id: 38. Already exists.
    3/4818  Done Processing Problem Id: 39. Already exists.
    4/4818  Done Processing Problem Id: 40. Already exists.
    5/4818  Done Processing Problem Id: 41. Already exists.
    6/4818  Done Processing Problem Id: 42. Already exists.
    7/4818  Done Processing Problem Id: 43. Already exists.
    8/4818  Done Processing Problem Id: 44. Already exists.
    9/4818  Done Processing Problem Id: 45. Already exists.
   10/4818  Done Processing Problem Id: 46. Already exists.
   11/4818  Done Processing Problem Id: 47. Already exists.
   12/4818  Done Processing Problem Id: 48. Already exists.
   13/4818  Done Processing Problem Id: 49. Already exists.
   14/4818  Done Processing Problem Id: 50. Already exists.
   15/4818  Done Processing Problem Id: 51. Already exists.
   16/4818  Done Processing Problem Id: 

### Finally Finshed!

Now, lets merge all these files into one file.

**Important** : Merging all problems into one files creates a csv file of size 120MB with over 5 million rows! So lets just merge a small amount of files.

In [5]:
mergedFile = path.join( problemSubFolder, 'merged2016Small.csv' )

if path.exists(mergedFile) is False:
    
    print ( "Merging CSV" )
    mergeSub = open( mergedFile, 'a' )

    for ind, problemId in enumerate ( subProblemIdList[:100] ):
        fileName = path.join( problemSubFolder, "{}.csv".format(problemId) )
        f = open(fileName)
        if ind > 0 : # Throw away the header file
            f.readline()
        for line in f:
            mergeSub.write(line)
        f.close()

    mergeSub.close()
        
submission = pd.read_csv(mergedFile)
display(submission.head())
submission.uname.describe()

,lan,mem,name,pid,rank,run,sbt,sid,uid,uname,ver
0,5,0,Mitsuaki TSUJI,36,-1,390,1461074093,17227551,839268,Mitsuaki.T,70
1,3,0,Mitsuaki TSUJI,36,-1,390,1461074395,17227578,839268,Mitsuaki.T,70
2,5,0,Mitsuaki TSUJI,36,-1,390,1461074419,17227581,839268,Mitsuaki.T,70
3,5,0,Mitsuaki TSUJI,36,-1,0,1461074419,17227582,839268,Mitsuaki.T,30
4,5,0,Mitsuaki TSUJI,36,-1,390,1461074513,17227591,839268,Mitsuaki.T,70


count      113319
unique      14473
top       vjudge1
freq         1451
Name: uname, dtype: object

# Convert it to Coding Submission History

In [12]:
codingHistory = pd.DataFrame()

verdicts = {
    10 : "Submission error",
    15 : " Can't be judged",
    20 : "In queue",
    30 : "Compile error",
    35 : "Restricted function",
    40 : "Runtime error",
    45 : "Output limit",
    50 : "Time limit",
    60 : "Memory limit",
    70 : "Wrong answer",
    80 : "PresentationE",
    90 : "Accepted"
}

languages = {
    1 : "C",
    2 : "Java",
    3 : "C++",
    4 : "Pascal",
    5 : "C++11",
    6 : "Python"
}

codingHistory['user_email'] = submission['uname'].map(lambda x : "{}@muktosoft.com".format(x))
codingHistory['time'] = submission['sbt'].map(lambda x: datetime.datetime.fromtimestamp(x) )
codingHistory['accepted'] = submission['ver'].map(lambda x: x is 90 )
codingHistory['exercise_id'] = submission['pid'].map(toUUID)
codingHistory['language'] = submission['lan'].map(lambda x: languages[x])
codingHistory['runtime'] = submission['run']
codingHistory['verdict'] = submission['ver'].map(lambda x: verdicts[x] )
codingHistory['sid'] = submission['sid']

codingHistory = codingHistory.sort_values(by="time").reset_index()

codingHistory.head()

,index,user_email,time,accepted,exercise_id,language,runtime,verdict,sid
0,63756,tpw284@muktosoft.com,2016-01-01 00:38:45,False,00000000-0000-0000-0000-000000000054,C++11,0,Wrong answer,16654590
1,101549,Neolink@muktosoft.com,2016-01-01 01:19:44,True,00000000-0000-0000-0000-000000000098,C++11,23,Accepted,16654658
2,63757,conchoda@muktosoft.com,2016-01-01 01:55:18,False,00000000-0000-0000-0000-000000000054,C++11,0,Wrong answer,16654750
3,63758,conchoda@muktosoft.com,2016-01-01 02:14:22,False,00000000-0000-0000-0000-000000000054,C++11,0,Wrong answer,16654795
4,91004,oculto@muktosoft.com,2016-01-01 02:37:37,False,00000000-0000-0000-0000-000000000083,C++,0,Runtime error,16654845


# Create Usage Log

In [30]:
%%cython

import numpy as np
cimport numpy as np

from __main__ import pd, codingHistory, datetime, toUUID, display


def genUsgeLog(startU, endU):
    """Generate usage_log from codingHistory
    
    You can send parameters to slice users
    
    @startU: start of slice
    @endU: end of slice
    """
    usageLog = pd.DataFrame()

    # Usage log will have user emails for each submission
    usageLog['user_email'] = codingHistory['user_email']
    usageLog['event_type'] = 'view'
    usageLog['current_item_id'] = codingHistory['exercise_id']
    usageLog['current_item_type'] = 'lesson'
    usageLog['current_enter_time'] = codingHistory['time']
    usageLog['is_completed'] = False
    usageLog['is_first_view'] = False
    
    users = codingHistory['user_email'].unique()[startU:endU]
    display(users)
    cdef int total = len(users)
    cdef int totalSub, prevIndex, loopInd
    cdef str sessionId, prevItemId
    
    newRows = [usageLog]
    timeDeltaMinus = pd.Timedelta(seconds=-1)
    timeDeltaPlus = pd.Timedelta(seconds=1)

    for unum, user in enumerate( users ):

        userSub = codingHistory[ codingHistory['user_email'] == user ]
        
        totalSub = len(userSub)
        print ( "Processing {:20} She has {} submissions. {:5}/{:<5}".format(user, totalSub, unum+1, total) )

        solved = set() # Contains solved problem by user
        seen = set()

        prevDate = datetime.date(2015,12,31)
        sessionId = toUUID(0)
        sessionStart = datetime.date(2015,12,31)
        prevItemId = ""
        prevItemType = ""
        prevEnterTime = ""
        prevIndex = -1
        
        loopInd = 0
        
        for index, sub in userSub.iterrows():
            loopInd = loopInd + 1
            #print("Processing Submissions {:5}/{:<5}".format(loopInd,totalSub) )

            eid = sub['exercise_id']
            today = sub['time'].to_pydatetime().date()
            delta = today - prevDate

            if delta.days > 0: # A new day has started
                # Create a session end event for previous session
                if prevIndex > -1:
                    newRow = usageLog[prevIndex:prevIndex+1].copy()
                    newRow['event_type'] = 'end'
                    newRows.append(newRow)
                
                prevDate = today
                sessionId = toUUID(sub['sid'])
                sessionStart = sub['time'] + timeDeltaMinus
                prevItemId = ""
                prevItemType = ""
                prevEnterTime = ""
                
                 # Create a session start event for this session
                newRow = usageLog[index:index+1].copy()
                newRow['event_type'] = 'start'
                newRow['session_id'] = sessionId
                newRow['session_start_time'] = sessionStart
                newRow['current_enter_time'] = newRow['session_start_time']
                newRows.append(newRow)

            usageLog.set_value(index,'session_id',sessionId)
            usageLog.set_value(index,'session_start_time',sessionStart)

            usageLog.set_value(index,'previous_item_id',prevItemId)
            usageLog.set_value(index,'previous_item_type',prevItemType)
            usageLog.set_value(index,'previous_enter_time',prevEnterTime)

            prevItemId = eid
            prevItemType = 'exercise'
            prevEnterTime = sub['time']

            if eid in seen:
                usageLog.set_value(index,'is_first_view',False)
            else:
                seen.add(eid)
                usageLog.set_value(index,'is_first_view',True)

            if sub['verdict'] == 'Accepted' or eid in solved:
                solved.add(eid)
                usageLog.set_value(index,'is_completed',True)
            else:
                usageLog.set_value(index,'is_completed',False)
                
            prevIndex = index
    
    usageLog = pd.concat(newRows,ignore_index=True)
    
    usageLog.loc[usageLog['event_type'] == 'end', 'current_enter_time'] += timeDeltaPlus
    
    usageLog = usageLog.sort_values(by='current_enter_time').reset_index(drop=True)
    usageLog = usageLog[['user_email','session_id','session_start_time','event_type','current_item_id',
                        'current_item_type','current_enter_time','is_completed','is_first_view',
                        'previous_item_id','previous_item_type','previous_enter_time']]
    return usageLog

usageLog = genUsgeLog(0,20)

array(['tpw284@muktosoft.com', 'Neolink@muktosoft.com',
       'conchoda@muktosoft.com', 'oculto@muktosoft.com',
       'acolla@muktosoft.com', 'dodecagon@muktosoft.com',
       'pachopepe@muktosoft.com', '01fate10@muktosoft.com',
       'archetype7@muktosoft.com', 'sleepplz@muktosoft.com',
       'szhu008@muktosoft.com', 'vjudge6@muktosoft.com',
       'vjudge7@muktosoft.com', 'vjudge3@muktosoft.com',
       'juhonkang@muktosoft.com', 'functionlyb@muktosoft.com',
       'Garrdy@muktosoft.com', 'rikka.chan.2k@muktosoft.com',
       'vjudge8@muktosoft.com', 'asadarshad_ar@muktosoft.com'], dtype=object)

Processing tpw284@muktosoft.com She has 4 submissions.     1/20   
Processing Neolink@muktosoft.com She has 1 submissions.     2/20   
Processing conchoda@muktosoft.com She has 7 submissions.     3/20   
Processing oculto@muktosoft.com She has 3 submissions.     4/20   
Processing acolla@muktosoft.com She has 1 submissions.     5/20   
Processing dodecagon@muktosoft.com She has 16 submissions.     6/20   
Processing pachopepe@muktosoft.com She has 1 submissions.     7/20   
Processing 01fate10@muktosoft.com She has 9 submissions.     8/20   
Processing archetype7@muktosoft.com She has 1 submissions.     9/20   
Processing sleepplz@muktosoft.com She has 1 submissions.    10/20   
Processing szhu008@muktosoft.com She has 1 submissions.    11/20   
Processing vjudge6@muktosoft.com She has 1423 submissions.    12/20   
Processing vjudge7@muktosoft.com She has 1406 submissions.    13/20   
Processing vjudge3@muktosoft.com She has 1429 submissions.    14/20   
Processing juhonkang@muktosoft.

In [15]:
%time usageLog = genUsgeLog(0,20)

Processing tpw284@muktosoft.com She has 4 submissions.     1/20   
Processing Neolink@muktosoft.com She has 1 submissions.     2/20   
Processing conchoda@muktosoft.com She has 7 submissions.     3/20   
Processing oculto@muktosoft.com She has 3 submissions.     4/20   
Processing acolla@muktosoft.com She has 1 submissions.     5/20   
Processing dodecagon@muktosoft.com She has 16 submissions.     6/20   
Processing pachopepe@muktosoft.com She has 1 submissions.     7/20   
Processing 01fate10@muktosoft.com She has 9 submissions.     8/20   
Processing archetype7@muktosoft.com She has 1 submissions.     9/20   
Processing sleepplz@muktosoft.com She has 1 submissions.    10/20   
Processing szhu008@muktosoft.com She has 1 submissions.    11/20   
Processing vjudge6@muktosoft.com She has 1423 submissions.    12/20   
Processing vjudge7@muktosoft.com She has 1406 submissions.    13/20   
Processing vjudge3@muktosoft.com She has 1429 submissions.    14/20   
Processing juhonkang@muktosoft.